In [1]:
#Importing Finviz API and neo4j drivers
from finvizfinance.quote import finvizfinance
from neo4j import GraphDatabase, RoutingControl

In [2]:
#Setting data_driver as main neo4j driver
data_driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "V3rYstr0ngp@sSw0rd"))

In [3]:
#Setting database name for validation and creation if needed
database_name = "fin-proj"

In [4]:
#Function to check if given database exists
def checkDatabase(db_name)->bool:
    result = data_driver.execute_query(
        "SHOW DATABASES yield name",
        database_="neo4j", routing_=RoutingControl.READ
    )
    for record in result.records:
        if record["name"] == db_name:
            return True
    
    return False

In [5]:
#Function to add a given database to the instancce
def addDatabase(db_name):
    data_driver.execute_query(
        "CREATE DATABASE $db_name",
        db_name = db_name,
        database_="neo4j",
        routing_=RoutingControl.WRITE    
    )

In [6]:
#Function to add given stock's details to the database
def addStock(db_name, stock_data):    
    #Formatting stock into readable key-value pairs in cypher
    stock_properties = ", ".join([f"`{key}` : $`{key}`" for key in stock_data.keys()])
    
    #Casting the data into a cypher query to add to the database
    add_query = f"CREATE (:stock {{{stock_properties}}})"
    
    #Executing the query to add stock details to the database
    data_driver.execute_query(add_query, stock_data, database_=db_name, routing_=RoutingControl.WRITE)

In [7]:
#Checking if stock is in database
def checkStock(db_name, ticker):
    #Storing result in res
    res, _, _ = data_driver.execute_query(
        "MATCH (n:stock {Ticker : $tick})"
        "RETURN n",
        tick=ticker,
        database_=db_name,
        routing_=RoutingControl.READ
    )
    
    if len(res):
        return True
    else:
        return False

In [8]:
#Converting Stock Ticker.txt into readable strings and adding to the database
def fileAdd(db_name, file_path):
    #Reading from given file for stock tickers
    f = open(file_path, "r")
    
    #Iterating through stock tickers in the file
    for stock in f:
        #Checking if stock already exists in the database
        if checkStock(db_name, stock[:-1]):
            continue
        
        #Adding the stock fundament to stock_data
        stock_data = finvizfinance(stock[:-1]).ticker_fundament()
        #Adding the ticker to stock_data
        stock_data['Ticker'] = stock[:-1]
        
        #Calling the function to execute the query to add the stock to the database
        addStock(db_name, stock_data)
        
    f.close()

In [9]:
def main():
    if not checkDatabase(database_name):
        addDatabase(database_name)
    
    fileAdd(database_name, "Stock tickers.txt")

In [ ]:
main()

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: Ticker)} {position: line: 1, column: 17, offset: 16} for query: 'MATCH (n:stock {Ticker : $tick})RETURN n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: stock)} {position: line: 1, column

: 